<a href="https://colab.research.google.com/github/Kim-Yeon-Jun/PyTorch/blob/main/pytorch_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 예제 : 문자RNN으로 성씨 생성하기
각 타임 스텝에서 RNN이 성씨에 포함될 수 있는 문자 집합에 대한 확률 분포를 계산한다는 의미\
성씨 생성 모델 2가지\
조건이 있는 SurnameGenerationModel : 국적 정보 사용O\
조건이 없는 SurnameGenerationModel : 국적 정보 사용X

In [ ]:
#시퀀스 예측 작업을 위한 SurnameDataset.__getitem__() 메서드
class SurnameDataset(Dataset):
  @classmethod
  def load_dataset_and_make_vetorizer(cls, surname_csv):
    """데이터셋을 로드하고 새로운 Vectorizer를 만듦
      매개변수 :
        surname_csv (str) : 데이터셋의 위치
      반환값 :
        SurnameDataset 객체
    """

    surname_df = pd.read_csv(surname_csv)
    return cls(surname_df, SurnameVecotrizer.from_dataframe(surname_df))

  def __getitem__(self,index):
    """ 파이토치 데이터셋의 주요 진입 메서드

    매개변수 :
      index(int) : 데이터 포인트에 대한 인덱스

    반환값 :
      데이터 포인트 (x_data, y_target, class_index)를 담은 딕셔너리

    """
    row = self._target_df.iloc[index]
    from_vector, to_vector = \
      self._vectorizer.vectorize(row.surname, self._max_seq_length)

    nationality_index = \
      self._vectorizer.nationality_vocab.lookup_token(row.nationality)

    return {'x_data' : from_vector,
            'y_target' : to_vector,
            'class_index' : nationality_index}

In [ ]:
#시퀀스 예측을 위한 SurnameVecotrizer.vectorizer()코드
class SurnameVectorizer(object):
    """ 어휘 사전을 생성하고 관리합니다 """

    def vectorize(self, surname, vector_length=-1):
        """ 성씨를 샘플과 타깃 벡터로 변환합니다
        성씨 벡터를 두 개의 벡터 surname[:-1]와 surname[1:]로 나누어 출력합니다.
        각 타임스텝에서 첫 번째 벡터가 샘플이고 두 번째 벡터가 타깃입니다.

        매개변수:
            surname (str): 벡터로 변경할 성씨
            vector_length (int): 인덱스 벡터의 길이를 맞추기 위한 매개변수
        반환값:
            튜플: (from_vector, to_vector)
                from_vector (numpy.ndarray): 샘플 벡터
                to_vector (numpy.ndarray): 타깃 벡터 vector
        """
        indices = [self.char_vocab.begin_seq_index]
        indices.extend(self.char_vocab.lookup_token(token) for token in surname)
        indices.append(self.char_vocab.end_seq_index)

        if vector_length < 0:
            vector_length = len(indices) - 1

        from_vector = np.empty(vector_length, dtype=np.int64)
        from_indices = indices[:-1]
        from_vector[:len(from_indices)] = from_indices
        from_vector[len(from_indices):] = self.char_vocab.mask_index

        to_vector = np.empty(vector_length, dtype=np.int64)
        to_indices = indices[1:]
        to_vector[:len(to_indices)] = to_indices
        to_vector[len(to_indices):] = self.char_vocab.mask_index

        return from_vector, to_vector

    @classmethod
    def from_dataframe(cls, surname_df):
        """데이터셋 데이터프레임으로 객체를 초기화합니다

        매개변수:
            surname_df (pandas.DataFrame): 성씨 데이터셋
        반환값:
            SurnameVectorizer 객체
        """
        char_vocab = SequenceVocabulary()
        nationality_vocab = Vocabulary()

        for index, row in surname_df.iterrows():
            for char in row.surname:
                char_vocab.add_token(char)
            nationality_vocab.add_token(row.nationality)

        return cls(char_vocab, nationality_vocab)

In [ ]:
#모델 1 : 조건이 없는 SurnameGenerationModel
class SurnameGenerationModel(nn.Module):
    def __init__(self, char_embedding_size, char_vocab_size, rnn_hidden_size,
                 batch_first=True, padding_idx=0, dropout_p=0.5):
        """
        매개변수:
            char_embedding_size (int): 문자 임베딩 크기
            char_vocab_size (int): 임베딩될 문자 개수
            rnn_hidden_size (int): RNN의 은닉 상태 크기
            batch_first (bool): 0번째 차원이 배치인지 시퀀스인지 나타내는 플래그
            padding_idx (int): 텐서 패딩을 위한 인덱스;
                torch.nn.Embedding를 참고하세요
            dropout_p (float): 드롭아웃으로 활성화 출력을 0으로 만들 확률
        """
        super(SurnameGenerationModel, self).__init__()

        self.char_emb = nn.Embedding(num_embeddings=char_vocab_size,
                                     embedding_dim=char_embedding_size,
                                     padding_idx=padding_idx)

        self.rnn = nn.GRU(input_size=char_embedding_size,
                          hidden_size=rnn_hidden_size,
                          batch_first=batch_first)

        self.fc = nn.Linear(in_features=rnn_hidden_size,
                            out_features=char_vocab_size)

        self._dropout_p = dropout_p

    def forward(self, x_in, apply_softmax=False):
        """모델의 정방향 계산

        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, input_dim)입니다.
            apply_softmax (bool): 소프트맥스 활성화를 위한 플래그로 훈련시에는 False가 되어야 합니다.
        반환값:
            결과 텐서. tensor.shape는 (batch, char_vocab_size)입니다.
        """
        x_embedded = self.char_emb(x_in)

        y_out, _ = self.rnn(x_embedded)

        batch_size, seq_size, feat_size = y_out.shape
        y_out = y_out.contiguous().view(batch_size * seq_size, feat_size)

        y_out = self.fc(F.dropout(y_out, p=self._dropout_p))

        if apply_softmax:
            y_out = F.softmax(y_out, dim=1)

        new_feat_size = y_out.shape[-1]
        y_out = y_out.view(batch_size, seq_size, new_feat_size)

        return y_out

In [ ]:
#모델 2 : 조건이 있는 SurnameGenerationModel
class SurnameGenerationModel(nn.Module):
    def __init__(self, char_embedding_size, char_vocab_size, num_nationalities,
                 rnn_hidden_size, batch_first=True, padding_idx=0, dropout_p=0.5):
        """
        매개변수:
            char_embedding_size (int): 문자 임베딩 크기
            char_vocab_size (int): 임베딩될 문자 개수
            rnn_hidden_size (int): RNN의 은닉 상태 크기
            batch_first (bool): 0번째 차원이 배치인지 시퀀스인지 나타내는 플래그
            padding_idx (int): 텐서 패딩을 위한 인덱스;
                torch.nn.Embedding를 참고하세요
            dropout_p (float): 드롭아웃으로 활성화 출력을 0으로 만들 확률
        """
        super(SurnameGenerationModel, self).__init__()

        self.char_emb = nn.Embedding(num_embeddings=char_vocab_size,
                                     embedding_dim=char_embedding_size,
                                     padding_idx=padding_idx)

        self.nation_emb = nn.Embedding(num_embeddings=num_nationalities,
                                       embedding_dim=rnn_hidden_size)

        self.rnn = nn.GRU(input_size=char_embedding_size,
                          hidden_size=rnn_hidden_size,
                          batch_first=batch_first)

        self.fc = nn.Linear(in_features=rnn_hidden_size,
                            out_features=char_vocab_size)

        self._dropout_p = dropout_p

    def forward(self, x_in, nationality_index, apply_softmax=False):
        """모델의 정방향 계산

        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, max_seq_size)입니다.
            nationality_index (torch.Tensor): 각 데이터 포인트를 위한 국적 인덱스
                RNN의 은닉 상태를 초기화하는데 사용합니다.
            apply_softmax (bool): 소프트맥스 활성화를 위한 플래그로 훈련시에는 False가 되어야 합니다.
        반환값:
            결과 텐서. tensor.shape는 (batch, char_vocab_size)입니다.
        """
        x_embedded = self.char_emb(x_in)

        # hidden_size: (num_layers * num_directions, batch_size, rnn_hidden_size)
        nationality_embedded = self.nation_emb(nationality_index).unsqueeze(0)

        y_out, _ = self.rnn(x_embedded, nationality_embedded)

        batch_size, seq_size, feat_size = y_out.shape
        y_out = y_out.contiguous().view(batch_size * seq_size, feat_size)

        y_out = self.fc(F.dropout(y_out, p=self._dropout_p))

        if apply_softmax:
            y_out = F.softmax(y_out, dim=1)

        new_feat_size = y_out.shape[-1]
        y_out = y_out.view(batch_size, seq_size, new_feat_size)

        return y_out

In [ ]:
#3차원 텐서와 시퀀스 손실 계산
def normalize_sizes(y_pred, y_true):
  """텐서 크기 정규화

  매개변수 :
    y_pred (torch.Tensor) : 모델의 출력
      3차원 텐서이면 행렬로 변환함
    y_true(torch.Tensor) : 타깃 예측
      행렬이명 벡터로 변환함

  """
  if len(y_pred.size()) == 3 :
    y_pred = y_pred.contiguous().view(-1, y_pred.size(2))
  if len(y_true.size()) == 2 :
    y_true = y_true.contiguous().view(-1)
  return y_pred, y_true

def sequence_loss(y_pred, y_true, mask_index):
  y_pred, y_true = normalize_size(y_pred, y_true)
  return F.cross_entropy(y_pred, y_true, ignore_index = mask_index)

In [ ]:
#성씨 생성을 위한 파라미터
args = Namespace(
    # 날짜와 경로 정보
    surname_csv="data/surnames/surnames_with_splits.csv",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="model_storage/ch7/model2_conditioned_surname_generation",
    # 모델 하이퍼파라미터
    char_embedding_size=32,
    rnn_hidden_size=32,
    # 훈련 하이퍼파라미터
    seed=1337,
    learning_rate=0.001,
    batch_size=128,
    num_epochs=100,
    early_stopping_criteria=5,
    # 실행 옵션
    catch_keyboard_interrupt=True,
    cuda=True,
    expand_filepaths_to_save_dir=True,
    reload_from_files=False,
)

In [ ]:
#조건이 없는 생성 모델의 샘플링
def sample_from_model(model, vectorizer, num_samples=1, sample_size=20,
                      temperature=1.0):
    """모델이 만든 인덱스 시퀀스를 샘플링합니다.

    매개변수:
        model (SurnameGenerationModel): 훈련 모델
        vectorizer (SurnameVectorizer): SurnameVectorizer 객체
        num_samples (int): 샘플 개수
        sample_size (int): 샘플의 최대 길이
        temperature (float): 무작위성 정도
            0.0 < temperature < 1.0 이면 최대 값을 선택할 가능성이 높습니다
            temperature > 1.0 이면 균등 분포에 가깝습니다
    반환값:
        indices (torch.Tensor): 인덱스 행렬
        shape = (num_samples, sample_size)
    """
    begin_seq_index = [vectorizer.char_vocab.begin_seq_index
                       for _ in range(num_samples)]
    begin_seq_index = torch.tensor(begin_seq_index,
                                   dtype=torch.int64).unsqueeze(dim=1)
    indices = [begin_seq_index]
    h_t = None

    for time_step in range(sample_size):
        x_t = indices[time_step]
        x_emb_t = model.char_emb(x_t)
        rnn_out_t, h_t = model.rnn(x_emb_t, h_t)
        prediction_vector = model.fc(rnn_out_t.squeeze(dim=1))
        probability_vector = F.softmax(prediction_vector / temperature, dim=1)
        indices.append(torch.multinomial(probability_vector, num_samples=1))
    indices = torch.stack(indices).squeeze().permute(1, 0)
    return indices

In [ ]:
#샘플링된 인덱스를 성씨 문자열로 매핑합니다
def decode_samples(sampled_indices, vectorizer):
    """인덱스를 성씨 문자열로 변환합니다

    매개변수:
        sampled_indices (torch.Tensor): `sample_from_model` 함수에서 얻은 인덱스
        vectorizer (SurnameVectorizer): SurnameVectorizer 객체
    """
    decoded_surnames = []
    vocab = vectorizer.char_vocab

    for sample_index in range(sampled_indices.shape[0]):
        surname = ""
        for time_step in range(sampled_indices.shape[1]):
            sample_item = sampled_indices[sample_index, time_step].item()
            if sample_item == vocab.begin_seq_index:
                continue
            elif sample_item == vocab.end_seq_index:
                break
            else:
                surname += vocab.lookup_index(sample_item)
        decoded_surnames.append(surname)
    return decoded_surnames

In [ ]:
#조건이 없는 모델에서 샘플링하기
samples = sample_from_model(unconditioned_model, vectorizer, num_samples=10)
decode_samples(samples, vectorizer)

In [ ]:
#조건이 있는 모델의 샘플링
def sample_from_model(model, vectorizer, nationalities, sample_size=20,
                      temperature=1.0):
    """모델이 만든 인덱스 시퀀스를 샘플링합니다.

    매개변수:
        model (SurnameGenerationModel): 훈련 모델
        vectorizer (SurnameVectorizer): SurnameVectorizer 객체
        nationalities (list): 국적을 나타내는 정수 리스트
        sample_size (int): 샘플의 최대 길이
        temperature (float): 무작위성 정도
            0.0 < temperature < 1.0 이면 최대 값을 선택할 가능성이 높습니다
            temperature > 1.0 이면 균등 분포에 가깝습니다
    반환값:
        indices (torch.Tensor): 인덱스 행렬
        shape = (num_samples, sample_size)
    """
    num_samples = len(nationalities)
    begin_seq_index = [vectorizer.char_vocab.begin_seq_index
                       for _ in range(num_samples)]
    begin_seq_index = torch.tensor(begin_seq_index,
                                   dtype=torch.int64).unsqueeze(dim=1)
    indices = [begin_seq_index]
    nationality_indices = torch.tensor(nationalities, dtype=torch.int64).unsqueeze(dim=0)
    h_t = model.nation_emb(nationality_indices)

    for time_step in range(sample_size):
        x_t = indices[time_step]
        x_emb_t = model.char_emb(x_t)
        rnn_out_t, h_t = model.rnn(x_emb_t, h_t)
        prediction_vector = model.fc(rnn_out_t.squeeze(dim=1))
        probability_vector = F.softmax(prediction_vector / temperature, dim=1)
        indices.append(torch.multinomial(probability_vector, num_samples=1))
    indices = torch.stack(indices).squeeze().permute(1, 0)
    return indices

In [ ]:
#조건이 있는 SurnameGenerationModel의 샘플링
for index in range(len(vectorizer.nationality_vocab)):
  nationality = vectorizer.nationality_vocab.lookup_index(index)

  print("{} 샘플 :".format(nationality))

  sampled_indices = sample_from_model(model = conditioned_model,
                                      vectorizer=vectorizer,
                                      nationalities=[index] *3
                                      temperature = 0.7)
  for sampled_surname in decode_samples(sampled_indices,
                                        vectorizer):
    print("- " + sampled_surname)